In [1]:
import os
import pandas as pd
import seaborn as sns
from scipy.stats import norm
import math as mt
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
import warnings
import itertools
import datetime
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
import pandas as pd
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
sns.set()
import warnings
warnings.filterwarnings('ignore')
path_store='C:/Users/chuna/Downloads/week5/store.csv'
path_train='C:/Users/chuna/Downloads/week5/train.csv'
path_test='C:/Users/chuna/Downloads/week5/test.csv'
isFile = os.path.isfile(path_train) 
print(isFile)                           

df_store = pd.read_csv(path_store)
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)



df_train=df_train.sample(frac =.000005)


True


In [3]:
df_train["Date"]= pd.to_datetime(df_train["Date"]) 
df_train["Date"]=df_train["Date"].astype('datetime64[D]')
holydates=df_train.loc[((df_train['StateHoliday'] != '0'))]['Date'].unique()


In [4]:
def nearest(pivot):
    return min(holydates, key=lambda x: abs(x - pivot))

In [5]:
def convert2int(df_train):

    df_train['Store'] = df_train['Store'].astype(int)
    df_train['DayOfWeek'] = df_train['DayOfWeek'].astype(int)
    
    
    df_train['Open'] = df_train['Open'].astype(int)
    df_train['Promo'] = df_train['Promo'].astype(int)
    df_train['SchoolHoliday'] = df_train['SchoolHoliday'].astype(int)
    return df_train

In [6]:
def convert2date(df_train):

    df_train["Date"]= pd.to_datetime(df_train["Date"]) 
    df_train["Date"]=df_train["Date"].astype('datetime64[D]')
    holydates=df_train.loc[((df_train['StateHoliday'] != '0'))]['Date'].unique()
    return df_train

In [7]:
def new_insights(df_train):

    df_train['day'] = df_train['Date'].dt.day_name() 
    df_train['month'] = df_train['Date'].dt.month_name() 
    df_train['year'] = df_train['Date'].dt.year  
    df_train.loc[((df_train['day'] == 'Saturday') |(df_train['day'] == 'Sunday')  ) , 'weekday'] = 'weekend' 
    df_train['weekday']=df_train['weekday'].fillna('weekday')
    df_train['StateHoliday'] = df_train['StateHoliday'].astype(str)
    


    df_train['to_holiday'] = df_train['Date'].apply(nearest)

    df_train['Difference_to_StateHoliday'] = df_train['Date'].sub(df_train['to_holiday'], axis=0)

    return df_train

In [8]:
def new_insights2(df_train):

    resort=df_train.copy()
    resort['Difference_to_StateHoliday'] = resort['Difference_to_StateHoliday'].astype(str)

    resort["Difference_to_StateHoliday"]= resort["Difference_to_StateHoliday"].str.split("d", 1)
    resort['Difference_to_StateHoliday'] = resort["Difference_to_StateHoliday"].str.get(0) 
    resort.sample(3)
    resort['Difference_to_StateHoliday'] = resort['Difference_to_StateHoliday'].astype(str)

    resort['Difference_to_StateHoliday'] =resort.Difference_to_StateHoliday.mask(resort.Difference_to_StateHoliday.str[-1].isin(['-','+']), resort.Difference_to_StateHoliday.str[-1].str.cat(resort.Difference_to_StateHoliday.str[:-1])).astype('float')


    resort['Difference_to_StateHoliday'] = resort['Difference_to_StateHoliday'].astype(int)
    df_train=resort
    return df_train

In [9]:

# read the training data set
data = df_train

# top rows of the data
data.head()

# seperate the independent and target variables
train_x = data.drop(columns=['Sales'])
train_y = data['Sales']

In [10]:
# import the BaseEstimator
from sklearn.base import BaseEstimator

# define the class OutletTypeEncoder
# This will be our custom transformer that will create 3 new binary columns
# custom transformer must have methods fit and transform
class OutletTypeEncoder(BaseEstimator):

    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        
        x_dataset=convert2int(x_dataset)
        x_dataset=convert2date(x_dataset)
        x_dataset=new_insights(x_dataset)
        x_dataset=new_insights2(x_dataset)
        

        return x_dataset

In [11]:
df_train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
330611,875,1,2014-09-22,3547,368,1,0,0,0
351455,84,6,2014-08-30,11410,1249,1,0,0,0
831026,22,7,2013-06-16,0,0,0,0,0,0
209658,39,6,2015-01-24,1898,278,1,0,0,0
433569,620,7,2014-06-08,0,0,0,0,0,0


In [12]:
# pre-processsing step
# Drop the columns - 
# Impute the missing values in column Item_Weight by mean
# Scale the data in the column Item_MRP
pre_process = ColumnTransformer(remainder='passthrough',
                                transformers=[('drop_columns', 'drop', ['Date',
                                                                        'Customers' ]),
                                                                        
                                                                      
                                              
                                              ('scale_data', StandardScaler(),['Sales']),
                                               ('encoder', OneHotEncoder(), ['Promo','StateHoliday','SchoolHoliday','day','month','weekday','Store','year']) ])
                                             

In [13]:
# Define the Pipeline
"""
Step1: get the oultet binary columns
Step2: pre processing
Step3: Train a Random Forest Model
"""
model_pipeline = Pipeline(steps=[('get_outlet_binary_columns', OutletTypeEncoder()), 
                                 ('pre_processing',pre_process),
                                 ('random_forest', RandomForestRegressor(max_depth=10,random_state=2))
                                 ])
# fit the pipeline with the training data
model_pipeline.fit(train_x,train_y)

# predict target values on the training data
model_pipeline.predict(train_x)

ValueError: A given column is not a column of the dataframe